In [1]:
# import random

# rgen = random.Random()

# def invert(x, y):
#     return pow(x, -1, y)

# class SievePolynomial:
#     def __init__(self, N, M, params, seed=None):
#         """ 多項式の初期化
#         """
#         from math import sqrt
#         if seed is not None:
#             rgen.seed(seed)
#         approx_val = sqrt(2*N) / M
#         best_a, best_q, best_ratio = None, None, None
#         start = params.idx_1000 or 0
#         end = params.idx_5000 or len(params) - 1
#         for _ in range(50):
#             a = 1
#             q = []
#             while a < approx_val:
#                 rand_p = 0
#                 while rand_p == 0 or rand_p in q:
#                     rand_p = rgen.randint(start, end)
#                 p = params.primes[rand_p]
#                 a *= p
#                 q.append(rand_p)
#             ratio = a / approx_val
#             if best_ratio is None or abs(ratio - 1) < abs(best_ratio - 1):
#                 best_q = q
#                 best_a = a
#                 best_ratio = ratio
    
#         a = best_a
#         q = best_q
    
#         B = []
#         for val in q:
#             q_l = params.primes[val]
#             gamma = params.sqrts[val] * invert(a // q_l, q_l) % q_l
#             if 2 * gamma > q_l:
#                 gamma = q_l - gamma
#             B.append(a//q_l*gamma)
        
#         b = sum(B)
#         self.B = B
#         self.modified_coeff = [a*a, 2*a*b, b*b - N]
#         self.a = a
#         self.b = b
#         self.a_inv = [None] * len(params)
#         self.b_ainv = [None] * len(params)
#         self.soln1 = [None] * len(params)
#         self.soln2 = [None] * len(params)
    
#         for idx, (prime, sqrts, logp) in enumerate(params):
#             if a % prime == 0:
#                 continue
#             self.a_inv[idx] = invert(a, prime)
#             self.b_ainv[idx] = [2*b_elem*self.a_inv[idx] % prime for b_elem in B]
#             self.soln1[idx] = (self.a_inv[idx]*(sqrts - b)) % prime
#             self.soln2[idx] = (self.a_inv[idx]*(-sqrts - b)) % prime

#     def update(self):
#         """ 多項式の更新
#         ref _initialize_ith_poly
        
#         N : number to be factored
#         factor_base : factor_base primes
#         i : integer denoting ith polynomial
#         g : (i - 1)th polynomial
#         B : array that stores a//q_l*gamma
#         """
#         from sympy.functions.elementary.integers import ceiling
#         from sympy.ntheory import trailing
#         v = trailing(i) + 1
#         if ceiling(i / (2**v)) % 2 == 1:
#             neg_pow = -1
#         else:
#             neg_pow = 1
#         b = g.b + 2*neg_pow*B[v - 1]
#         a = g.a
#         g = SievePolynomial([a*a, 2*a*b, b*b - N], a, b)
#         for fb in factor_base:
#             if a % fb.prime == 0:
#                 continue
#             fb.soln1 = (fb.soln1 - neg_pow*fb.b_ainv[v - 1]) % fb.prime
#             fb.soln2 = (fb.soln2 - neg_pow*fb.b_ainv[v - 1]) % fb.prime
        
#     def eval(self, x):
#         """ 多項式の評価
#         """
#         ans = 0
#         for coeff in self.modified_coeff:
#             ans *= x
#             ans += coeff
#         return ans

In [2]:
import math
import random

rgen = random.Random()

def invert(x, y):
    return pow(x, -1, y)

class SievePolynomial:
    """ 多項式を提供する
    """
    def __init__(self, N, M, params, seed=None):
        """ 多項式の初期化
        
        N : 分解する整数
        M : 篩長
        params : QuadraticSieveBasicParameter
        seed : 乱数のシード
        
        self.B
        self.modified_coeff
        self.a
        self.b
        self.a_inv
        self.b_ainv
        self.soln1
        self.soln2
        
        """
        from math import sqrt
        if seed is not None:
            rgen.seed(seed)
        approx_val = sqrt(2*N) / M
        best_a, best_q, best_ratio = None, None, None
        start = params.idx_1000 or 0
        end = params.idx_5000 or len(params) - 1
        for _ in range(50):
            a = 1
            q = []
            while a < approx_val:
                rand_p = 0
                while rand_p == 0 or rand_p in q:
                    rand_p = rgen.randint(start, end)
                p = params.primes[rand_p]
                a *= p
                q.append(rand_p)
            ratio = a / approx_val
            if best_ratio is None or abs(ratio - 1) < abs(best_ratio - 1):
                best_q = q
                best_a = a
                best_ratio = ratio
    
        a = best_a
        q = best_q
    
        B = []
        for val in q:
            q_l = params.primes[val]
            gamma = params.sqrts[val] * invert(a // q_l, q_l) % q_l
            if 2 * gamma > q_l:
                gamma = q_l - gamma
            B.append(a//q_l*gamma)
        
        b = sum(B)
        self.B = B
        self.modified_coeff = [a*a, 2*a*b, b*b - N]
        self.a = a
        self.b = b
        self.a_inv = [None] * len(params)
        self.b_ainv = [None] * len(params)
        self.soln1 = [None] * len(params)
        self.soln2 = [None] * len(params)
    
        for idx, (prime, sqrts, logp) in enumerate(params):
            if a % prime == 0:
                continue
            self.a_inv[idx] = invert(a, prime)
            self.b_ainv[idx] = [2*b_elem*self.a_inv[idx] % prime for b_elem in B]
            self.soln1[idx] = (self.a_inv[idx]*(sqrts - b)) % prime
            self.soln2[idx] = (self.a_inv[idx]*(-sqrts - b)) % prime
            
            
    
    def update(self, i):
        """Initialization stage of ith poly. After we finish sieving 1`st polynomial
        here we quickly change to the next polynomial from which we will again
        start sieving. Suppose we generated ith sieve polynomial and now we
        want to generate (i + 1)th polynomial, where ``1 <= i <= 2**(j - 1) - 1``
        where `j` is the number of prime factors of the coefficient `a`
        then this function can be used to go to the next polynomial. If
        ``i = 2**(j - 1) - 1`` then go to _initialize_first_polynomial stage.
    
        Parameters
        ==========
    
        N : number to be factored
        factor_base : factor_base primes
        i : integer denoting ith polynomial
        g : (i - 1)th polynomial
        B : array that stores a//q_l*gamma
        """
        from sympy.functions.elementary.integers import ceiling
        v = 1
        j = i
        while(j % 2 == 0):
            v += 1
            j //= 2
        if ceiling(i / (2**v)) % 2 == 1:
            neg_pow = -1
        else:
            neg_pow = 1
        b = self.b + 2*neg_pow*self.B[v - 1]
        a = self.a
        g = SievePolynomial([a*a, 2*a*b, b*b - N], a, b)
        for fb in factor_base:
            if a % fb.prime == 0:
                continue
            fb.soln1 = (fb.soln1 - neg_pow*fb.b_ainv[v - 1]) % fb.prime
            fb.soln2 = (fb.soln2 - neg_pow*fb.b_ainv[v - 1]) % fb.prime
    
        return g
        
    def eval(self, x):
        """ 多項式の評価
        """
        ans = 0
        for coeff in self.modified_coeff:
            ans *= x
            ans += coeff
        return ans

class QuadraticSieveBasicParameter:
    """ 2次篩のためのパラメータ管理を提供する
    """
    def __init__(self, n, prime_bound):
        """ パラメータの設定
        
        n : 分解対象の整数
        prime_bound : 素数の閾値 これより小さい素数を使う
        
        nはprime_boundより小さい素因数を持たないとする
        prime_bound > 3. 実際はもっと大きくしないと意味ないと思う
        
        self.primes : 使用する素数
        self.sqrts : a**2 = n (mod prime) を満たすa
        self.logp : 篩のための概算値
        self.idx_1000 : primeが初めて1000を超えるindex
        self.idx_5000 : primeが初めて5000を超えるindex
        
        """
        from sympy.ntheory.generate import sieve
        from sympy.ntheory.residue_ntheory import jacobi_symbol as jacobi, _sqrt_mod_prime_power
        self.primes = [2]
        self.sqrts = [1]
        self.logp = [round(math.log(2)*2**10)]
        self.idx_1000 = None
        self.idx_5000 = None
        for prime in sieve.primerange(3, prime_bound):
            if jacobi(n, prime) == 1:
                if prime > 1000 and self.idx_1000 is None:
                    self.idx_1000 = len(self) - 1
                if prime > 5000 and self.idx_5000 is None:
                    self.idx_5000 = len(self) - 1
                self.primes.append(prime)
                self.sqrts.append(_sqrt_mod_prime_power(n, prime, 1)[0])
                self.logp.append(round(math.log(prime)*2**10))

    def __len__(self):
        """ 行列の要素になる数
        """
        return len(self.primes)
    
    def __iter__(self):
        """ 設定値を順次返す
        """
        for idx in range(len(self)):
            yield self.primes[idx], self.sqrts[idx], self.logp[idx]

    def get_sieve_array(self, M):
        pass
    
    def check_smoothness(self, num):
        """ numがsmoothかを判定する
        
        (list[int], bool) : ベクトルとスムースか？
        (None, None) の場合
        """
        from sympy.ntheory.factor_ import multiplicity
        vec = [0] * len(self)
        for idx in range(len(self)):
            p = self.primes[idx]
            r = multiplicity(p, num)
            if r:
                vec[idx] = r % 2
                num //= p**r
        if num == 1:
            return [0] + vec, True
        if num == -1:
            return [1] + vec, True
        if isprime(num):
            return num, False
        return None, None

In [ ]:
class PolyManager:
    def __init__(self, n, prime_bound):
        self.n = n
        self._set_params(prime_bound)
    
    def _set_params(self, prime_bound):
        """ パラメータの設定
        
        n : 分解対象の整数
        prime_bound : 素数の閾値 これより小さい素数を使う
        
        nはprime_boundより小さい素因数を持たないとする
        prime_bound > 3. 実際はもっと大きくしないと意味ないと思う
        
        self.n : 対象の整数
        self.primes : 使用する素数
        self.sqrts : a**2 = n (mod prime) を満たすa
        self.logp : 篩のための概算値
        self.idx_1000 : primeが初めて1000を超えるindex
        self.idx_5000 : primeが初めて5000を超えるindex
        
        """
        from sympy.ntheory.generate import sieve
        from sympy.ntheory.residue_ntheory import jacobi_symbol as jacobi, _sqrt_mod_prime_power
        self.primes = [2]
        self.sqrts = [1]
        self.logp = [round(math.log(2)*2**10)]
        self.idx_1000 = None
        self.idx_5000 = None
        for prime in sieve.primerange(3, prime_bound):
            if jacobi(self.n, prime) == 1:
                if prime > 1000 and self.idx_1000 is None:
                    self.idx_1000 = len(self) - 1
                if prime > 5000 and self.idx_5000 is None:
                    self.idx_5000 = len(self) - 1
                self.primes.append(prime)
                self.sqrts.append(_sqrt_mod_prime_power(self.n, prime, 1)[0])
                self.logp.append(round(math.log(prime)*2**10))
                
    def _init_poly(self, M, seed=None):
        """ 多項式の初期化
        
        M : 篩長
        seed : 乱数のシード
        
        self.B
        self.modified_coeff
        self.a
        self.b
        self.a_inv
        self.b_ainv
        self.soln1
        self.soln2
        
        """
        from math import sqrt
        if seed is not None:
            rgen.seed(seed)
        approx_val = sqrt(2*self.n) / M
        best_a, best_q, best_ratio = None, None, None
        start = self.idx_1000 or 0
        end = self.idx_5000 or len(self) - 1
        for _ in range(50):
            a = 1
            q = []
            while a < approx_val:
                rand_p = 0
                while rand_p == 0 or rand_p in q:
                    rand_p = rgen.randint(start, end)
                p = self.primes[rand_p]
                a *= p
                q.append(rand_p)
            ratio = a / approx_val
            if best_ratio is None or abs(ratio - 1) < abs(best_ratio - 1):
                best_q = q
                best_a = a
                best_ratio = ratio
    
        a = best_a
        q = best_q
    
        B = []
        for val in q:
            q_l = self.primes[val]
            gamma = self.sqrts[val] * invert(a // q_l, q_l) % q_l
            if 2 * gamma > q_l:
                gamma = q_l - gamma
            B.append(a//q_l*gamma)
        
        b = sum(B)
        self.B = B
        self.modified_coeff = [a*a, 2*a*b, b*b - N]
        self.a = a
        self.b = b
        self.a_inv = [None] * len(params)
        self.b_ainv = [None] * len(params)
        self.soln1 = [None] * len(params)
        self.soln2 = [None] * len(params)
        
        for idx in range(len(self)):
            
    
        for idx, (prime, sqrts, logp) in enumerate(params):
            if a % prime == 0:
                continue
            self.a_inv[idx] = invert(a, prime)
            self.b_ainv[idx] = [2*b_elem*self.a_inv[idx] % prime for b_elem in B]
            self.soln1[idx] = (self.a_inv[idx]*(sqrts - b)) % prime
            self.soln2[idx] = (self.a_inv[idx]*(-sqrts - b)) % prime
    
    def __len__(self):
        """ 行列の要素になる数
        """
        return len(self.primes)

In [16]:
def qs(N, prime_bound, M, checked=True):
    """
    M : 篩長
    checked : prime_boundより小さい素因数がないことは確認済みであるか。
              Falseの場合は、この関数の中でチェックする。
    """
    from sympy.ntheory import sieve
    proper_factor = set()
    if not checked:
        for p in sieve.primerange(prime_bound):
            if N % p == 0:
                proper_factor.add(p)
                while N % p == 0:
                    N //= p
    # パラメータの決定。ランダム要素なし
    params = QuadraticSieveBasicParameter(N, prime_bound)
    # 多項式の無限提供
    while True:
        poly = SievePolynomial(N, M, params)
        for p in poly:
            sieve_array = _gen_sieve_array(M, factor_base)
            s_rel, p_f = _trial_division_stage(N, M, factor_base, sieve_array, ith_sieve_poly, partial_relations, ERROR_TERM)
    return proper_factor